toxim
deeptox

kaggel par bhi he ek 
https://github.com/masashitsubaki/molecularGNN_smiles

# Resnet vs MobileNet 

In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optim
import tensorflow.keras.models as models
import tensorflow.keras.preprocessing as prepro
import tensorflow.keras.losses as loss
from tensorflow_addons.losses import SigmoidFocalCrossEntropy as focal_loss

import cv2
import numpy as np 
import pandas as pd 
import json

from tqdm.auto import trange

In [2]:
DATA = r"C:\Users\suyash\Desktop\deep fake\train_sample_videos"
JSON = r"C:\Users\suyash\Desktop\deep fake\train_sample_videos\metadata.json"

data = open(JSON)
dataset = json.load(data)
data = []
for d in dataset:
    data.append(d)

In [3]:
ytrain = []
for d in dataset.values():
    for s in d.values():
        if s == "FAKE":
            ytrain.append(0)
        else:
            ytrain.append(1)
        break
y = np.array(ytrain)

In [4]:
np.array(y).shape

(400,)

In [5]:
no_video = len(data)
video = []
ytrain = []
cond = True
load = trange(no_video, desc=f"Loading:0/{no_video} || Progress", unit="images", ncols=1000)
for l in load:
    cap = cv2.VideoCapture(DATA+"/"+data[l])
    v = []
    e = 0 
    if y[l] == 1 or l%3 == 0:
        cond = True
    else:
        cond = False
        
    while cond:
        ret, frame = cap.read() 
        e=e+1
        if not ret:
            break
        if e%5==0:
            frame = cv2.resize(frame, (224, 224), interpolation = cv2.INTER_AREA)/255.0
            v.append(frame)
        if e>250:
            break
            
    if cond:
        v = np.array(v)
        video.append(v)
        ytrain.append(y[l])

    load.set_description(f"Loading:{l+1}/{no_video} || Progress", refresh=True)
video = np.array(video)
ytrain = np.array(ytrain)
video.shape   

Loading:0/400 || Progress:   0%|                                                                              …

(184, 50, 224, 224, 3)

In [6]:
ytrain = np.array(ytrain)

In [7]:
type(video), type(ytrain)

(numpy.ndarray, numpy.ndarray)

# Model 

In [8]:
def ConvLSTMBlock(x, filters, kernel=3, seq=True):
    x = layers.ConvLSTM2D(filters, kernel, padding="same", return_sequences=seq)(x)
    x = layers.BatchNormalization(momentum=0.5)(x)
    x = layers.ReLU()(x)
    if seq:
        x = layers.MaxPool3D((1,2,2))(x)
    else:
        x = layers.MaxPool2D((2,2))(x)
    return x

In [32]:
def ConvLSTM(inp, filters):
    inp = layers.Input(inp)
    x = ConvLSTMBlock(inp, filters)
    x = ConvLSTMBlock(x, filters*2)
    x = ConvLSTMBlock(x, filters*4)
    x = ConvLSTMBlock(x, filters*8)
    x = ConvLSTMBlock(x, filters*16)
    x = layers.Reshape((50, 7*7*256))(x)
    x = layers.LSTM(256, activation="relu", return_sequences=True)(x)
    x = layers.LSTM(128, activation="relu")(x)
    x = layers.Dense(10, activation="relu")(x)
    x = layers.Dense(1, activation="sigmoid")(x)
    
    model = models.Model(inputs=inp, outputs=x, name="deepfake")
    return model

In [33]:
model = ConvLSTM((50, 224, 224, 3), 16)
model.summary()

Model: "deepfake"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 50, 224, 224, 3)  0         
                             ]                                   
                                                                 
 conv_lstm2d_53 (ConvLSTM2D)  (None, 50, 224, 224, 16)  11008    
                                                                 
 batch_normalization_53 (Bat  (None, 50, 224, 224, 16)  64       
 chNormalization)                                                
                                                                 
 re_lu_53 (ReLU)             (None, 50, 224, 224, 16)  0         
                                                                 
 max_pooling3d_52 (MaxPoolin  (None, 50, 112, 112, 16)  0        
 g3D)                                                            
                                                          

In [34]:
model.compile(optimizer=optim.Adam(lr=0.001), loss=loss.BinaryCrossentropy(), metrics="accuracy" )
model.fit(video[:150], ytrain[:150], batch_size=5, epochs=3, verbose=1, validation_data=(video[150:], ytrain[150:]), validation_batch_size=5)

c:\Users\suyash\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/3


: 

: 

In [ ]:
model.save("./meop.h5")